In [ ]:
import pandas as pd
import numpy as np

def classify_return_zone(df):
    x = df['x_coord']
    y = df['y_coord']
    sign = x * y  

    # Calculate court thirds
    court_width_units = 324 * 38.2764654418 / 12  
    third_width = court_width_units / 3         
    left_bound = -third_width / 2           
    right_bound = third_width / 2        

    # Determine zone
    if x < left_bound:
        zone = 'left'
    elif left_bound <= x <= right_bound:
        zone = 'middle'
    elif x > right_bound:
        zone = 'right'
    else:
        zone = np.nan

    if sign > 0:
        side = 'ad'
    elif sign < 0:
        side = 'deuce'
    else:
        side = np.nan

    # Type Return
    if side == 'deuce':
        if zone == 'left':
            direction = 'crosscourt'
        elif zone == 'middle':
            direction = 'middle'
        elif zone == 'right':
            direction = 'down the line'
        else:
            direction = np.nan
    elif side == 'ad':
        if zone == 'left':
            direction = 'down the line'
        elif zone == 'middle':
            direction = 'middle'
        elif zone == 'right':
            direction = 'crosscourt'
        else:
            direction = np.nan
    else:
        direction = np.nan

    return pd.Series({'side': side, 'placement': direction})

# Load file
player_name = 'Rudy Quan'
file_path = f'../../../data/mens/{player_name}/combined.xlsx'
df = pd.read_excel(file_path, sheet_name='Shots')

def return_direction_bar(df, player_name):
    # Keep Player's Returns (Shot == 2)
    returns = df[(df['Player'] == player_name) & (df['Shot'] == 2)].copy()

    # Transform Coordinates
    returns.loc[:, 'x_coord'] = returns['Hit (x)'] * 38.2764654418
    returns.loc[:, 'y_coord'] = (returns['Hit (y)'] - 11.8872) * 38.2764654418

    # Create "side" and "placement" columns
    returns[['side', 'placement']] = returns.apply(classify_return_zone, axis=1)

    # Get counts
    counts = returns[['side', 'placement']].value_counts().reset_index(name='count')


    output = {'returns': [
        {
            'side': 'deuce',
            'placements': {'crosscourt': 0, 'middle': 0, 'down the line': 0}
        },
        {
            'side': 'ad',
            'placements': {'crosscourt': 0, 'middle': 0, 'down the line': 0}
        }
    ]}

    for _, row in counts.iterrows():
        side = row['side']
        placement = row['placement']
        count = int(row['count'])
        for side_dict in output['returns']:
            if side_dict['side'] == side:
                side_dict['placements'][placement] = count


    return output, counts

In [5]:
return_data, counts_df = return_direction_bar(df, player_name)
print(return_data)
print(counts_df)

{'returns': [{'side': 'deuce', 'placements': {'crosscourt': 39, 'middle': 275, 'down the line': 29}}, {'side': 'ad', 'placements': {'crosscourt': 41, 'middle': 232, 'down the line': 41}}]}
    side      placement  count
0  deuce         middle    275
1     ad         middle    232
2     ad     crosscourt     41
3     ad  down the line     41
4  deuce     crosscourt     39
5  deuce  down the line     29
